In [1]:
from visualization.scripts.quick_loader import get_event
from visualization.scripts.plotting import plot_xyz_plotly, plot_xyz_plotly_3d
from utility.EverythingNeeded import get_memory_size_MB

In [11]:
df_node, df_edge, batch = get_event(
    # input_dir='/Users/avencast/PycharmProjects/trkgnn/workspace/bepc.magnet/Tracker_GNN.root',
    input_dir='/Users/avencast/CLionProjects/darkshine-simulation/workspace/Tracker_GNN.root',
    collection='DigitizedRecTrk',
    index=27
)

print(f'{get_memory_size_MB(batch):.3f} MB')

plot_xyz_plotly(df_node, df_edge, threshold=0.5, no_predict=True)

0.007 MB


In [ ]:
# plot_xyz_plotly_3d(df_node, df_edge, threshold=0.1, no_predict=True)

In [ ]:
from visualization.scripts.plotting import read_local_csv, visual_summary_log

In [ ]:
df, t = read_local_csv('/Users/avencast/PycharmProjects/trkgnn/workspace/bl0/rec.4.csv')
fig = visual_summary_log(df, t)
fig.show()

In [ ]:
df, t = read_local_csv('/Users/avencast/PycharmProjects/trkgnn/workspace/bl0/rec.csv')
fig = visual_summary_log(df, t)
fig.show()

In [ ]:
df